### Melon 100 Chart 스크래핑
* 100곡의 노래의 제목과 Song ID 파싱
* Song ID로 상세페이지로 100번 요청을 해서 상세정보를 파싱
* 100곡의 상세정보를 json file에 저장
* json file을 표데이터를 만들기
* 표데이터를 DB에 저장

In [1]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
req_header = {
    #browser 정보
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
}
res = requests.get(url, headers=req_header)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select("div#tb_list"))) #1개
    print(len(soup.select("div#tb_list tr"))) #101개
    print(len(soup.select("div#tb_list tr div.wrap_song_info"))) #200개
    print(len(soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']"))) #100개
    
    atag_list = soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']")
    
    #100곡의 Song 정보를 저장할 List 선언
    song_list = []
    
    for idx,atag in enumerate(atag_list,1):
        #1곡의 Song 정보를 저장할 Dict 선언
        song_dict = {}
        
        title = atag.text
        href = atag['href']
        
        matched = re.search(r'(\d+)\);', href)
        song_id = None
        if matched:
            song_id = matched.group(1)
        
        song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        #print(f'{idx}위, {title} {song_url}')
        
        song_dict['title'] = title    
        song_dict['song_id'] = song_id
        song_dict['song_url'] = song_url
        
        song_list.append(song_dict)

print(len(song_list))
song_list[0:3]

200
1
101
200
100
100


[{'title': 'Savage',
  'song_id': '34041584',
  'song_url': 'https://www.melon.com/song/detail.htm?songId=34041584'},
 {'title': 'STAY',
  'song_id': '33658563',
  'song_url': 'https://www.melon.com/song/detail.htm?songId=33658563'},
 {'title': 'My Universe',
  'song_id': '33998510',
  'song_url': 'https://www.melon.com/song/detail.htm?songId=33998510'}]

In [7]:
import requests
from bs4 import BeautifulSoup
import re

req_header = {
    #browser 정보
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
}

song_detail_list = []
for idx,song in enumerate(song_list[0:3],1):
    url = song['song_url']
    res = requests.get(url, headers=req_header)
    print(idx, res.status_code)
    
    song_detail_dict = {}
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        song_detail_dict['곡명'] = song['title']
        
        span_tag = soup.select("a[href*='goArtistDetail'] span")
        if span_tag:
            song_detail_dict['가수'] = span_tag[0].text
         
        dd_tag = soup.select("div.meta dd")
        if dd_tag:    
            song_detail_dict['앨범'] = dd_tag[0].text
            song_detail_dict['발매일'] = dd_tag[1].text
            song_detail_dict['장르'] = dd_tag[2].text
        
        song_id = song['song_id']
        like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
        like_res = requests.get(like_url, headers=req_header)
        #print(like_res.status_code)    
        #print(like_res.json()['contsLike'][0]['SUMMCNT'])
        like_cnt = like_res.json()['contsLike'][0]['SUMMCNT']
        song_detail_dict['좋아요'] = like_cnt
        
    print(song_detail_dict)

1 200
{'곡명': 'Savage', '가수': 'aespa', '앨범': 'Savage - The 1st Mini Album', '발매일': '2021.10.05', '장르': '댄스', '좋아요': 51031}
2 200
{'곡명': 'STAY', '가수': 'The Kid LAROI', '앨범': 'Stay', '발매일': '2021.07.09', '장르': 'POP', '좋아요': 165131}
3 200
{'곡명': 'My Universe', '가수': 'Coldplay', '앨범': 'My Universe', '발매일': '2021.09.24', '장르': 'POP', '좋아요': 104742}
